In [1]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
app_dir = '/Users/nikhilvs/repos/nyu/covid-19'

In [2]:
df = pd.read_csv(os.path.join(app_dir, 'data/staging/unified.csv'))
us_df = df[df['country_region'] == 'United States']

In [3]:
df.dtypes

country_region                                         object
date                                                   object
retail_and_recreation_percent_change_from_baseline    float64
grocery_and_pharmacy_percent_change_from_baseline     float64
parks_percent_change_from_baseline                    float64
transit_stations_percent_change_from_baseline         float64
workplaces_percent_change_from_baseline               float64
residential_percent_change_from_baseline              float64
C1_School closing                                     float64
C2_Workplace closing                                  float64
C3_Cancel public events                               float64
C4_Restrictions on gatherings                         float64
C5_Close public transport                             float64
C6_Stay at home requirements                          float64
C7_Restrictions on internal movement                  float64
C8_International travel controls                      float64
E1_Incom

In [4]:
TRAIN_COLUMNS = [
    # 'country_region',
    # 'date',
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline',
    'C1_School closing',
    'C2_Workplace closing',
    'C3_Cancel public events',
    'C4_Restrictions on gatherings',
    'C5_Close public transport',
    'C6_Stay at home requirements',
    'C7_Restrictions on internal movement',
    'C8_International travel controls',
    'E1_Income support',
    'E2_Debt/contract relief',
    'H1_Public information campaigns',
    'H2_Testing policy',
    'H3_Contact tracing',
    'StringencyIndex'
]

TARGET_COLUMNS = [
    'cases'
]

In [6]:
train_subset = us_df[us_df['date'] <= '2020-05-05']
test_subset = us_df[us_df['date'] > '2020-05-05']

scaler = preprocessing.StandardScaler().fit()
X_train = scaler.transform(train_subset[TRAIN_COLUMNS])
y_train = train_subset[TARGET_COLUMNS]
X_test = scaler.transform(test_subset[TRAIN_COLUMNS])
y_test = test_subset[TARGET_COLUMNS]

# Naive linear regression - no shift in number of cases
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)
pred = reg.predict(X_test)
print(mean_squared_error(y_test, pred))
print(r2_score(y_test, pred))

NameError: name 'X_train' is not defined

In [78]:
for SHIFT_IMPACT_DAYS in list(range(1, 10)):
    us_shifted_df = us_df.copy()

    SHIFT_COL = 'cases_' + str(SHIFT_IMPACT_DAYS)
    us_shifted_df[SHIFT_COL] = us_shifted_df['cases'].shift(-SHIFT_IMPACT_DAYS)
    us_shifted_df = us_shifted_df.dropna()
    train_subset = us_shifted_df[us_shifted_df['date'] <= '2020-04-20']
    test_subset = us_shifted_df[us_shifted_df['date'] > '2020-04-20']

    X_train = preprocessing.scale(train_subset[TRAIN_COLUMNS])
    y_train = train_subset[SHIFT_COL]
    X_test = preprocessing.scale(test_subset[TRAIN_COLUMNS])
    y_test = test_subset[SHIFT_COL]

    reg = LinearRegression().fit(X_train, y_train)
    reg.score(X_train, y_train)
    pred = reg.predict(X_test)
    print(mean_squared_error(y_test, pred))
    print(r2_score(y_test, pred))

843958898706.8376
-40.8527550870816
841960956520.6454
-46.58236466902283
841586989794.7777
-53.80130008678709
840956248106.7882
-61.89873091310629
841175136389.8099
-71.15624907036523
841560305127.0378
-82.38497935644682
839371233423.5619
-97.42785905729639
834069660206.705
-118.19003851826028
828626072298.9722
-147.69993170002178


In [17]:
train_subset = us_df[us_df['date'] <= '2020-05-05']
test_subset = us_df[us_df['date'] > '2020-05-05']

X_train = train_subset[TRAIN_COLUMNS]
y_train = train_subset[TARGET_COLUMNS]
X_test = test_subset[TRAIN_COLUMNS]
y_test = test_subset[TARGET_COLUMNS]

poly = PolynomialFeatures(4)

X_train_poly = poly.fit_transform(X_train)
scaler = preprocessing.StandardScaler().fit(X_train_poly)

X_train_poly = scaler.transform(X_train_poly)
X_test_poly = scaler.transform(poly.fit_transform(X_test))

reg = LinearRegression().fit(X_train_poly, y_train)
reg.score(X_train_poly, y_train)
pred = reg.predict(X_test_poly)
print(mean_squared_error(y_test, pred))
print(r2_score(y_test, pred))

105638719090.81653
-117.01288131993083


(81, 231)